In [ ]:
import asyncio
from ircclient import IRCClient
from hojong_py_util.util import parse_url

# twitch IRC server info
SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

#'''
from random import random

# freenode irc server example info
SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)
PASS = None
#'''

# callback to print chat
def print_chat(nick, chat):
    print('*** [RECEIVED] %s: %s' % (nick, chat))

# callback mysql
def log_to_mysql(nick, chat):
    import pymysql as mysql
    
    HOST = 'localhost'
    USER = 'user'
    PASS = 'pass'
    DB = 'db'
    
    conn = mysql.connect(host=HOST, user=USER,password=PASS,db=DB,charset='utf8mb4',cursorclass=mysql.cursors.DictCursor)
    cur = conn.cursor()
    sql = 'SELECT * FROM test_tb'
    cur.execute(sql)

# callback mongodb
def log_to_mongodb(nick, chat):
    raise Exception('not implemented')
    
    import pymongo
    
    HOST = 'localhost'
    PORT = 27017
    
    cli = pymongo.MongoClient(host=HOST, port=PORT)
    db = cli.testdb
    coll = db.testcoll
    print(coll.insert_one({nick: chat}).inserted_id)
    coll.find_one()

def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** [SETTING_INFO] %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
    # if you want to use other callback, change the callback parameter
    client = IRCClient(CHANNEL, NICK, PASS, print_chat, loop)
    coroutine = loop.create_connection(lambda: client, host, port)

    try:
        loop.run_until_complete(coroutine) # connect
        # bot send chatting
        client.send_chat('hello, I am a bot!')
        loop.run_forever() # data receiving
    except KeyboardInterrupt as e:
        print(e)
    finally:
        loop.close()
    
if __name__ == '__main__':
    main()